In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

In [2]:
df = pd.DataFrame({
    'key1': ['a', 'a', 'b', 'b', 'a'],
    'key2': ['one', 'two', 'one', 'two', 'one'],
    'data1': np.random.randn(5),
    'data2': np.random.randn(5)
})
df

,data1,data2,key1,key2
0,-1.647531,1.202193,a,one
1,-0.595163,-0.506052,a,two
2,0.107433,1.704062,b,one
3,-0.502813,1.605012,b,two
4,-0.125855,0.402592,a,one


In [3]:
k1_means = df.groupby('key1').mean().add_prefix('mean_')
k1_means

,mean_data1,mean_data2
key1,,
a,-0.789516,0.366245
b,-0.197690,1.654537


In [4]:
pd.merge(df, k1_means, left_on='key1', right_index=True)

,data1,data2,key1,key2,mean_data1,mean_data2
0,-1.647531,1.202193,a,one,-0.789516,0.366245
1,-0.595163,-0.506052,a,two,-0.789516,0.366245
4,-0.125855,0.402592,a,one,-0.789516,0.366245
2,0.107433,1.704062,b,one,-0.197690,1.654537
3,-0.502813,1.605012,b,two,-0.197690,1.654537


In [5]:
people = pd.DataFrame(np.random.randn(5, 5),
                     columns=['a', 'b', 'c', 'd', 'e'],
                     index=['Joe', 'Steve', 'Wes', 'Jim', 'Travis'])
people.loc[2:3, ['b', 'c']] = np.nan
people

,a,b,c,d,e
Joe,0.551921,-1.318736,-1.436388,0.565507,-0.307591
Steve,0.677492,0.868982,0.347102,0.747336,-0.366783
Wes,-1.140157,NaN,NaN,-1.336293,-0.428658
Jim,-1.733966,0.328241,1.710569,-1.804403,-0.512140
Travis,0.737260,1.013736,0.369393,-0.363434,1.422262


In [6]:
key = ['one', 'two', 'one', 'two', 'one']
people.groupby(key).mean()

,a,b,c,d,e
one,0.049675,-0.152500,-0.533498,-0.378074,0.228671
two,-0.528237,0.598611,1.028835,-0.528533,-0.439462


In [7]:
people.groupby(key).transform(np.mean)

,a,b,c,d,e
Joe,0.049675,-0.152500,-0.533498,-0.378074,0.228671
Steve,-0.528237,0.598611,1.028835,-0.528533,-0.439462
Wes,0.049675,-0.152500,-0.533498,-0.378074,0.228671
Jim,-0.528237,0.598611,1.028835,-0.528533,-0.439462
Travis,0.049675,-0.152500,-0.533498,-0.378074,0.228671


In [8]:
def demean(arr):
    return arr - arr.mean()

demeaned = people.groupby(key).transform(demean)
demeaned

,a,b,c,d,e
Joe,0.502247,-1.166236,-0.902890,0.943581,-0.536262
Steve,1.205729,0.270370,-0.681734,1.275870,0.072679
Wes,-1.189832,NaN,NaN,-0.958220,-0.657329
Jim,-1.205729,-0.270370,0.681734,-1.275870,-0.072679
Travis,0.687585,1.166236,0.902890,0.014639,1.193591


In [9]:
demeaned.groupby(key).mean()

,a,b,c,d,e
one,0.0,0.000000e+00,0.000000e+00,-3.700743e-17,0.0
two,0.0,2.775558e-17,-1.110223e-16,0.000000e+00,0.0


In [10]:
tips = pd.read_csv('../plot/tips.csv')
tips['tip_pct'] = tips['tip'] / tips['total_bill']
tips[:6]

,total_bill,tip,sex,smoker,day,time,size,tip_pct
0,16.99,1.01,Female,No,Sun,Dinner,2,0.059447
1,10.34,1.66,Male,No,Sun,Dinner,3,0.160542
2,21.01,3.50,Male,No,Sun,Dinner,3,0.166587
3,23.68,3.31,Male,No,Sun,Dinner,2,0.139780
4,24.59,3.61,Female,No,Sun,Dinner,4,0.146808
5,25.29,4.71,Male,No,Sun,Dinner,4,0.186240


In [11]:
def top(df, n=5, column='tip_pct'):
    return df.sort_values(by=column)[-n:]

top(tips, n=6)

,total_bill,tip,sex,smoker,day,time,size,tip_pct
109,14.31,4.00,Female,Yes,Sat,Dinner,2,0.279525
183,23.17,6.50,Male,Yes,Sun,Dinner,4,0.280535
232,11.61,3.39,Male,No,Sat,Dinner,2,0.291990
67,3.07,1.00,Female,Yes,Sat,Dinner,1,0.325733
178,9.60,4.00,Female,Yes,Sun,Dinner,2,0.416667
172,7.25,5.15,Male,Yes,Sun,Dinner,2,0.710345


In [12]:
tips.groupby('smoker').apply(top)

total_bill   tip     sex smoker   day    time  size   tip_pct
smoker                                                                   
No     88        24.71  5.85    Male     No  Thur   Lunch     2  0.236746
       185       20.69  5.00    Male     No   Sun  Dinner     5  0.241663
       51        10.29  2.60  Female     No   Sun  Dinner     2  0.252672
       149        7.51  2.00    Male     No  Thur   Lunch     2  0.266312
       232       11.61  3.39    Male     No   Sat  Dinner     2  0.291990
Yes    109       14.31  4.00  Female    Yes   Sat  Dinner     2  0.279525
       183       23.17  6.50    Male    Yes   Sun  Dinner     4  0.280535
       67         3.07  1.00  Female    Yes   Sat  Dinner     1  0.325733
       178        9.60  4.00  Female    Yes   Sun  Dinner     2  0.416667
       172        7.25  5.15    Male    Yes   Sun  Dinner     2  0.710345

In [13]:
tips.groupby(['smoker', 'day']).apply(top, n=1, column='total_bill')

total_bill    tip     sex smoker   day    time  size  \
smoker day                                                              
No     Fri  94        22.75   3.25  Female     No   Fri  Dinner     2   
       Sat  212       48.33   9.00    Male     No   Sat  Dinner     4   
       Sun  156       48.17   5.00    Male     No   Sun  Dinner     6   
       Thur 142       41.19   5.00    Male     No  Thur   Lunch     5   
Yes    Fri  95        40.17   4.73    Male    Yes   Fri  Dinner     4   
       Sat  170       50.81  10.00    Male    Yes   Sat  Dinner     3   
       Sun  182       45.35   3.50    Male    Yes   Sun  Dinner     3   
       Thur 197       43.11   5.00  Female    Yes  Thur   Lunch     4   

                  tip_pct  
smoker day                 
No     Fri  94   0.142857  
       Sat  212  0.186220  
       Sun  156  0.103799  
       Thur 142  0.121389  
Yes    Fri  95   0.117750  
       Sat  170  0.196812  
       Sun  182  0.077178  
       Thur 197  0.115982

In [14]:
result = tips.groupby('smoker')['tip_pct'].describe()
result

,count,mean,std,min,25%,50%,75%,max
smoker,,,,,,,,
No,151.0,0.159328,0.039910,0.056797,0.136906,0.155625,0.185014,0.291990
Yes,93.0,0.163196,0.085119,0.035638,0.106771,0.153846,0.195059,0.710345


In [15]:
result.unstack('smoker')

       smoker
count  No        151.000000
       Yes        93.000000
mean   No          0.159328
       Yes         0.163196
std    No          0.039910
       Yes         0.085119
min    No          0.056797
       Yes         0.035638
25%    No          0.136906
       Yes         0.106771
50%    No          0.155625
       Yes         0.153846
75%    No          0.185014
       Yes         0.195059
max    No          0.291990
       Yes         0.710345
dtype: float64

In [16]:
tips.groupby('smoker', group_keys=False).apply(top)

,total_bill,tip,sex,smoker,day,time,size,tip_pct
88,24.71,5.85,Male,No,Thur,Lunch,2,0.236746
185,20.69,5.00,Male,No,Sun,Dinner,5,0.241663
51,10.29,2.60,Female,No,Sun,Dinner,2,0.252672
149,7.51,2.00,Male,No,Thur,Lunch,2,0.266312
232,11.61,3.39,Male,No,Sat,Dinner,2,0.291990
109,14.31,4.00,Female,Yes,Sat,Dinner,2,0.279525
183,23.17,6.50,Male,Yes,Sun,Dinner,4,0.280535
67,3.07,1.00,Female,Yes,Sat,Dinner,1,0.325733
178,9.60,4.00,Female,Yes,Sun,Dinner,2,0.416667
172,7.25,5.15,Male,Yes,Sun,Dinner,2,0.710345


In [17]:
frame = pd.DataFrame({'data1': np.random.randn(1000), 'data2': np.random.randn(1000)})
factor = pd.cut(frame.data1, 4)
factor[:10]

0      (0.431, 2.199]
1     (-1.337, 0.431]
2      (0.431, 2.199]
3    (-3.113, -1.337]
4    (-3.113, -1.337]
5      (0.431, 2.199]
6     (-1.337, 0.431]
7     (-1.337, 0.431]
8     (-1.337, 0.431]
9      (0.431, 2.199]
Name: data1, dtype: category
Categories (4, interval[float64]): [(-3.113, -1.337] < (-1.337, 0.431] < (0.431, 2.199] < (2.199, 3.968]]

In [18]:
def get_stats(group):
    return {'min': group.min(), 'max': group.max(), 'count': group.count(), 'mean': group.mean()}

grouped = frame.data2.groupby(factor)
grouped.apply(get_stats).unstack()

,count,max,mean,min
data1,,,,
"(-3.113, -1.337]",75.0,2.381312,-0.023901,-2.006120
"(-1.337, 0.431]",598.0,2.753955,-0.056107,-2.803141
"(0.431, 2.199]",316.0,2.464492,-0.024345,-2.944028
"(2.199, 3.968]",11.0,1.171605,-0.024929,-1.172823


In [19]:
grouping = pd.qcut(frame.data1, 10, labels=False)
grouped = frame.data2.groupby(grouping)
grouped.apply(get_stats).unstack()

,count,max,mean,min
data1,,,,
0,100.0,2.381312,-0.014854,-2.006120
1,100.0,2.753955,-0.032748,-2.803141
2,100.0,2.118613,-0.116574,-2.348472
3,100.0,2.186973,-0.137910,-1.655903
4,100.0,1.990565,0.006287,-2.644519
5,100.0,2.407433,0.025907,-2.035179
6,100.0,2.085852,-0.157911,-2.507681
7,100.0,2.081673,0.070475,-2.036779
8,100.0,2.464492,-0.048761,-2.944028


In [20]:
s = pd.Series(np.random.randn(6))
s[::2] = np.nan
s

0         NaN
1   -0.044287
2         NaN
3    0.459010
4         NaN
5   -0.330613
dtype: float64

In [21]:
s.fillna(s.mean())

0    0.028037
1   -0.044287
2    0.028037
3    0.459010
4    0.028037
5   -0.330613
dtype: float64

In [22]:
states = ['Ohio', 'New York', 'Vermont', 'Florida', 'Oregon', 'Nevada', 'California', 'Idaho']
group_key = ['East'] * 4 + ['West'] * 4
data = pd.Series(np.random.randn(8), index=states)
data[['Vermont', 'Nevada', 'Idaho']] = np.nan
data

Ohio          0.581970
New York      1.316077
Vermont            NaN
Florida      -0.141113
Oregon        2.114002
Nevada             NaN
California    0.703557
Idaho              NaN
dtype: float64

In [23]:
data.groupby(group_key).mean()

East    0.585645
West    1.408780
dtype: float64

In [24]:
fill_mean = lambda g: g.fillna(g.mean())
data.groupby(group_key).apply(fill_mean)

Ohio          0.581970
New York      1.316077
Vermont       0.585645
Florida      -0.141113
Oregon        2.114002
Nevada        1.408780
California    0.703557
Idaho         1.408780
dtype: float64

In [25]:
suits = ['H', 'S', 'C', 'D']
card_val = (list(range(1, 11)) + [10] * 3) * 4
base_names = ['A'] + list(range(2, 11)) + ['J', 'K', 'Q']
cards = []
for suit in ['H', 'S', 'C', 'D']:
    cards.extend(str(num) + suit for num in base_names)

deck = pd.Series(card_val, index=cards)
deck[:13]

AH      1
2H      2
3H      3
4H      4
5H      5
6H      6
7H      7
8H      8
9H      9
10H    10
JH     10
KH     10
QH     10
dtype: int64

In [26]:
def draw(deck, n=5):
    return deck.take(np.random.permutation(len(deck))[:n])

draw(deck)

6D     6
KC    10
AH     1
5C     5
9C     9
dtype: int64

In [27]:
get_suit = lambda card: card[-1]
deck.groupby(get_suit).apply(draw, n=2)

C  7C      7
   10C    10
D  AD      1
   8D      8
H  10H    10
   5H      5
S  4S      4
   5S      5
dtype: int64

In [28]:
deck.groupby(get_suit, group_keys=False).apply(draw, n=2)

QC    10
KC    10
KD    10
2D     2
JH    10
3H     3
6S     6
KS    10
dtype: int64

In [29]:
df = pd.DataFrame({'category': ['a', 'a','a', 'a', 'b', 'b', 'b', 'b'], 'data': np.random.randn(8), 'weights': np.random.rand(8)})
df

,category,data,weights
0,a,0.494255,0.818897
1,a,0.671223,0.621172
2,a,0.184245,0.597918
3,a,-2.012900,0.601998
4,b,1.436117,0.477723
5,b,0.811073,0.162309
6,b,-0.541589,0.964352
7,b,0.789406,0.266112


In [30]:
grouped = df.groupby('category')
get_wavg = lambda g: np.average(g['data'], weights=g['weights'])
grouped.apply(get_wavg)

category
a   -0.106027
b    0.270249
dtype: float64

In [31]:
close_px = pd.read_csv('stock_px.csv', parse_dates=True, index_col=0)
close_px.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2214 entries, 2003-01-02 to 2011-10-14
Data columns (total 4 columns):
AAPL    2214 non-null float64
MSFT    2214 non-null float64
XOM     2214 non-null float64
SPX     2214 non-null float64
dtypes: float64(4)
memory usage: 86.5 KB


In [32]:
close_px[-4:]

,AAPL,MSFT,XOM,SPX
2011-10-11,400.29,27.00,76.27,1195.54
2011-10-12,402.19,26.96,77.16,1207.25
2011-10-13,408.43,27.18,76.37,1203.66
2011-10-14,422.00,27.27,78.11,1224.58


In [33]:
rets = close_px.pct_change().dropna()
spx_corr = lambda x: x.corrwith(x['SPX'])
by_year = rets.groupby(lambda x: x.year)
by_year.apply(spx_corr)

,AAPL,MSFT,XOM,SPX
2003,0.541124,0.745174,0.661265,1.0
2004,0.374283,0.588531,0.557742,1.0
2005,0.467540,0.562374,0.631010,1.0
2006,0.428267,0.406126,0.518514,1.0
2007,0.508118,0.658770,0.786264,1.0
2008,0.681434,0.804626,0.828303,1.0
2009,0.707103,0.654902,0.797921,1.0
2010,0.710105,0.730118,0.839057,1.0
2011,0.691931,0.800996,0.859975,1.0


In [34]:
by_year.apply(lambda g: g['AAPL'].corr(g['MSFT']))

2003    0.480868
2004    0.259024
2005    0.300093
2006    0.161735
2007    0.417738
2008    0.611901
2009    0.432738
2010    0.571946
2011    0.581987
dtype: float64

In [35]:
def regress(data, yvar, xvars):
    Y = data[yvar]
    X = data[xvars]
    X['intercept'] = 1.
    result = sm.OLS(Y, X).fit()
    return result.params

by_year.apply(regress, 'AAPL', ['SPX'])

,SPX,intercept
2003,1.195406,0.000710
2004,1.363463,0.004201
2005,1.766415,0.003246
2006,1.645496,0.000080
2007,1.198761,0.003438
2008,0.968016,-0.001110
2009,0.879103,0.002954
2010,1.052608,0.001261
2011,0.806605,0.001514
